In [1]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

# Retrograde Analyse für das Schach-Endspiel

Ein Schachspiel ist gewonnen wenn, die gegnerische Figur matt gesetzt wurde. Ein Matt wird erreicht, wenn der Gegner
nach einem Zug, keine Möglichkeit hat seine Figuren so zu bewegen, dass sein König im nächsten Zug *nicht* geschlagen
wird.

Bei einer geringen Anzahl an Figuren $P$ im Spielzustand lassen sich alle möglichen Positionen und damit Züge berechnen.
Aus dieser Rechnung kann eine Strategie entwickelt werden den Gegner zu schlagen.

Folgend werden folgende Definitionen verwendet:
* $board.pieces$: Liste der Figuren, welche in einem Zustand vorhanden sind.
* $validBoards$: Alle Zustände des Schachspiels, die gegen keine Regeln verstoßen.
* $wonBoards$: Alle Zustände des Schachspiels, in denen ein Spieler gewonnen hat.
* $previousStates(b)$: Alle Zustände, aus denen durch Ausführen eines einzelnen Zuges der Zustand $b$ erreicht werden kann.

Seien alle möglichen (validen) Kombinationen von Positionen der Figuren $P$ die Menge $S$.  
Für $S$ gilt:  
$$
board \in S \implies \forall p \in P : p \in board.pieces \\
\land \\
board \in S \implies board \in validBoards
$$

Aus der Menge $S$ lassen sich Zustände auswählen, welche $n$ Züge vom Sieg entfernt sind. Diese Zustände lassen sich
in $S_n$ zusammenfassen.  
Für alle Zustände, in denen ein Spieler matt gesetzt ist, gilt:  
$$board \in S_0 \implies board \in wonBoards \land board \in S$$  
Aus dieser Definition können induktiv die verbleibenden $S_n$ hergeleitet werden:  
$$board \in S_{n+1} \iff board \in S \land \exists b \in S_n: board \in previousStates(b)$$ 

Dieses Notebook wird zur Berechnung der $S_n$ Mengen verwendet. Diese werden benötigt, um letztendlich ein Schach-Endspiel
lösen zu können. 

## Imports

Im Rahmen dieses Notebooks werden für die Darstellung und Durchführung der besagten Analyse folgende drei Bibliotheken 
verwendet:

* [``chess``](https://python-chess.readthedocs.io/en/latest/): Die Python-Schach-Bibliothek, mit welcher Spielzustände 
dargestellt werden, Züge und Zustände ausgewertet etc.
* [``clear_output``](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html) aus der 
IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen verwendet wird.
* [``pickle``](https://docs.python.org/3/library/pickle.html): Wird zum Speichern der Ergebnisse verwendet.
* [``os``](https://docs.python.org/3/library/os.html): Wird zum Löschen einer temporären Datei verwendet.

In [439]:
import chess
from IPython.display import clear_output
import json#import pickle
import os

## Funktionen zur Bestimmung aller gültigen Positionen

Ein Schachbrett besteht aus insgesamt acht Spalten und Zeilen. Die Spalten werden durch Buchstaben gekennzeichnet, 
die Zeilen wiederum durch Zahlen. Aus der Kombination einer Spalte (z.B. a) und einer Zahl (z.B. 1) erhält man eine 
eindeutige Kennzeichnung für ein Feld (z.B. a1).

Die folgende Funktion kombiniert die Buchstaben a bis h mit den Zahlen 1 bis 8 zu Feldnamen und gibt diese zurück.

In [440]:
def get_all_squares():
    columns = {
        1 : 'a',
        2 : 'b',
        3 : 'c',
        4 : 'd',
        5 : 'e',
        6 : 'f',
        7 : 'g',
        8 : 'h'
    }
    
    all_squares = []
    for row in range(1,9):
        for col_num in range(1,9):
            column = columns[col_num]
            all_squares.append(chess.parse_square(column + str(row)))
            
    return all_squares

Das Erstellen jeglicher Boards wird mit der Funktion ``place_piece_everywhere_on_every_board`` umgesetzt. 
Diese erhält folgende Parameter: 

* ``piece``: Die zu platzierende Figur, als Objekt der chess-Library
* ``list_of_boards``: Eine Liste mit Board-Objekten, auf welchen die Figur platziert werden soll

Die Funktion betrachtet jede Situation in der `list_of_boards` und platziert das `piece` auf jeden freien 
Platz in dieser. Jedes Mal, wenn eine Figur platziert wird, wird eine Kopie des Board-Objektes erstellt, 
die ``list_of_boards`` wird nicht verändert.  
Wenn der zweite König platziert wird, wird die Situation zusätzlich auf Validität überprüft.
Wenn alle Figuren platziert wurden, werden nur Boards, in denen Schwarz matt ist zurückgegeben. 


//TODO: Brauchen wir das? Stroetmann sagte ja das ist bei simplen Funktionen nicht nötig
* Über alle Spielbretter in der ``list_of_boards`` iterieren.
* Die bereits verwendeten Spielfelder speichern.
* Über alle Spielfelder iterieren.
* Wenn das Spielfeld noch nicht benutzt ist:
  * Das Spielbrett kopieren und die Figur auf diesem Feld platzieren.
  * Die Kopie der Ergebnisliste anfügen.

Die Funktion gibt als Ergebnis eine Liste aller generierten Zustände gespeichert in ``result_list`` zurück.

In [441]:
def place_figure_everywhere_on_every_board(piece, list_of_boards, piece_count):
    result_list = []
    all_squares = get_all_squares()
    
    for board in list_of_boards:
        squares_used = list(board.piece_map().keys())
        for square in all_squares:
            if square not in squares_used:
                tmp_board = board.copy()
                tmp_board.set_piece_at(square, piece)
                
                if len(squares_used) >= 1 and not tmp_board.is_valid(): 
                    # Don't process invalid boards further than the second king
                    continue
                    
                outcome = tmp_board.outcome()
                if outcome is not None:
                    if outcome.winner is not None and tmp_board.is_valid():
                        result_list.append(tmp_board)
                        continue
                        
                if len(squares_used) + 1 < piece_count: #Board is valid, but needs more pieces
                    result_list.append(tmp_board)
    return result_list

## Die Ursprungsmenge $S_0$ erstellen
Als Basis der Berechnung dient die Liste $S_0$. Diese enthält alle möglichen Konstellationen der Spielfiguren auf dem
Spielbrett, in denen Weiß Schwarz besiegt hat.
Hierfür werden die Figuren mit der Funktion ``place_figure_everywhere_on_every_board`` auf allen 
Positionen platziert.   

Die Funktion `setup_boards` übernimmt dies und gibt sowohl die Liste $S_0$ als auch eine Menge an Tupeln der Form
`(Bool, String)` zurück, welche verwendet wird, um effizient bereits verwendete Boards zu identifizieren.

In [442]:
def setup_boards(user_supplied_pieces):
    pieces_to_place = create_piece_list(user_supplied_pieces)
    
    empty_board = chess.Board().empty()
    empty_board.turn = chess.BLACK
    s_0 = [empty_board]
    
    piece_count = len(pieces_to_place)
    for piece in pieces_to_place:
        s_0 = place_figure_everywhere_on_every_board(piece, s_0, piece_count)
        
    used_boards = set()
    for board in s_0:
        used_boards.add((board.turn,board.__str__()))

    print(str(len(s_0)) + " Boards in S_0")
    return s_0, used_boards

# A queen will automatically be replaced by a pawn
def create_piece_list(user_supplied_pieces):
    if chess.Piece.from_symbol("P") in user_supplied_pieces:
        user_supplied_pieces.remove(chess.Piece.from_symbol("P"))
        user_supplied_pieces.append(chess.Piece.from_symbol("Q"))
        
    return [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k")] + user_supplied_pieces

Der nächste Schritt besteht darin sämtliche $S_{n}$ Mengen zu bestimmen. Hierzu werden die bereits bestimmten $S_n$
Mengen genommen und alle Bretter aus S selektiert, die innerhalb eines Zugs in $S_n$ landen. Die Umsetzung erfolgt 
durch die Funktion ``previous_states``. Diese erhält unter anderem zwei Listen, mit denen dieses Verfahren durchgeführt
werden soll. Alle Funktionsparameter können aus der nachfolgenden Liste entnommen werden:

* ``s_n``: Die Liste mit Board-Objekten, deren Spielbretter sich $n$ Züge vom Sieg entfernt befinden.
* ``s``: Die Liste, aus welcher die nächste Stufe ($n+1$) ausgewählt werden soll.
* ``s_white_ascii``: Liste der ASCII Representationen aller bereits zugeordneten Spielbretter mit weiß am Zug.
* ``iteration_count``: $m$ des $S_m$, welches gerade berechnet wird.

Der Algorithmus zur Bestimmung der Menge $S_{n+1}$ wird im folgenden Abschnit beschrieben.

* Für effizientere Vergleiche wird für jedes Objekt in $S$ die ASCII-Representation gespeichert.
* Über die Objekte in $S_n$ iterieren.
  * Den Spieler, welcher am Zug ist wechseln (Da, um im aktuellen Zustand anzukommen, der andere Spieler einen Zug
  gemacht hat)
  * Pseudo legale Spielzüge von der Bibliothek berechnen lassen:
    Ein pseudo legaler Spielzug ist ein Spielzug, welcher die grundsätzlichen Bewegungsregeln der Figur einhält,
    aber das Schachbrett unter Umständen in einen nicht regelkonformen Zustand versetzt.
  * Über diese Spielzüge iterieren.
    * Den Zug ausführen.
    * Überprüfen, ob das Board sich in einem erlaubten Zustand befindet.
      Wenn nein, den Schleifendurchlauf abbrechen.
    * Mittels der ASCII-Representation überprüfen, ob das modifizierte Spielbrett in $S$ gefunden wird.
    * Wenn es gefunden wird, die ASCII-Representation zu $s_{n+1-ASCII}$ hinzufügen.
    * Den Zug rückgängig machen.
  * Den Spieler, welcher ursprünglich am Zug war, wiederherstellen.
* Über $S$ iterieren.
  * Wenn sich das Spielbrett in $S_{n+1-ASCII}$ befindet, wird es $S_{n+1}$ hinzugefügt.
    * Besonderheit für Spielbretter, bei welchen Schwarz am Zug ist: Diese werden nur zu $S_{n+1}$ hinzugefügt, wenn
    alle Züge, welche von dieser Position möglich sind, in einem $S_m$ mit $m < n$ enden.
    Für diese Überprüfung wird $S_{white-ASCII}$ verwendet.
  * Wenn sich das Spielbrett nicht in $S_{n+1-ASCII}$ befindet, wird es $S_{tmp}$ hinzugefügt.
* Wurden Positionen mit Weiß am Zug berechnet, diese $S_{white-ASCII}$ hinzufügen.

Außerdem müssen bei der Durchführung des Algorithmus weitere Aspekte berücksichtig werden:

* Da Bauern nur in eine Richtung laufen können, müssen die rückwärts Schritte eines Bauern manuell durchgeführt werden. Bauern werden daher im ersten Schritt ignoriert.
* Bauern, die die oberste Reihe des Spielfeldes erreichen können zu einer anderen Figur eingetauscht werden. Dieser Schritt wird nicht durch die Pseudo-Legal-Moves abgedeckt, daher wird, sollte sich eine Königin in der obersten Reihe befinden, diese manuell durch einen Bauern ersetzt.

Sobald die Funktion ausgeführt wurde, wird ein Tupel bestehend aus den Mengen $S_{n+1}$, $S_{tmp}$ und $S_{white-ASCII}$ zurückgegeben.

In [443]:
def previous_states(s_n, used_boards, iteration_count, user_wants_pawn):
    #variables
    s_n1 = []
    s_n1_tuples = set()

    for i in range(len(s_n)):
        status = "Calculating S" + str(iteration_count) + " - Board " + str(i+1) + " of " + str(len(s_n)) + \
                     " from S" + str(iteration_count-1)
        clear_output(wait=True)
        print(status)

        # Copy current board and invert the player
        chess_board = s_n[i].copy()
        chess_board.turn = chess_board.turn ^ True

        # Find all Pawns
        pawn_positions = find_pawns(chess_board)
        
        # try moves and check if they lead to new boards
        tmp_list, tmp_set = regular_moves(chess_board, used_boards, s_n1_tuples, pawn_positions)
        s_n1 += tmp_list
        s_n1_tuples |= tmp_set
        
        if user_wants_pawn:
            # Push all pawns one row back and check if this leads to new boards
            if len(pawn_positions) > 0:
                tmp_list, tmp_set = pawn_moves(chess_board, used_boards, s_n1_tuples)
                s_n1 += tmp_list
                s_n1_tuples |= tmp_set
            
            # Exchange Queens with Pawns
            queen_positions = check_top_row_for_queen(chess_board)
            if queen_positions:
                tmp_list, tmp_set = replace_queen_with_pawn(chess_board, used_boards, s_n1_tuples, queen_positions)
                s_n1 += tmp_list
                s_n1_tuples |= tmp_set


        # Restore the original state of the board
        chess_board.turn = chess_board.turn ^ True

    # Only needed for Black-Moves
    if iteration_count % 2 == 0:
        clear_output(wait=True)
        print("Calculating S" + str(iteration_count) + " - Checking Black Moves for determinism")
        s_n1, s_n1_tuples = check_black_determinism(s_n1, used_boards)


    clear_output(wait=True)
    print("Done with S" + str(iteration_count))

    return s_n1, used_boards | s_n1_tuples

## Hilfsfunktionen für die Berechnung

In [444]:
def regular_moves(chess_board, used_boards, s_n1_tuples, pawn_positions):
    new_boards = []
    new_tuples = set()
    
    for pLMove in chess_board.pseudo_legal_moves:
            if chess.square_name(pLMove.from_square) not in pawn_positions:
                chess_board.push(pLMove)
                chess_board.turn = chess_board.turn ^ True
                if not chess_board.is_valid():
                    chess_board.turn = chess_board.turn ^ True
                    chess_board.pop()
                    continue
                # If the new board is found in S, it can be reached in one step
                tuple_rep = (chess_board.turn,chess_board.__str__())
                if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples:
                #if tuple_rep not in known_boards:
                    new_boards.append(chess_board.copy())
                    new_tuples.add(tuple_rep)
                chess_board.turn = chess_board.turn ^ True
                chess_board.pop()
    
    return new_boards, new_tuples


'''
* As pawns only move forward there is no backwards-step in the pseudo legal moves
* to mitigate this the pawn is moved back one row manually
'''
def pawn_moves(chess_board, used_boards, s_n1_tuples):
    new_boards = []
    new_tuples = set()
    
    for pawn in chess_board.pieces(chess.PAWN, True):
        if chess_board.piece_at(pawn - 9) is None:
            chess_board.remove_piece_at(pawn)
            chess_board.set_piece_at(pawn - 9, chess.Piece.from_symbol('P'))
            if chess_board.is_valid():
                tuple_rep = (chess_board.turn,chess_board.__str__())
                if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples:
                    new_boards.append(chess_board.copy())
                    new_tuples.add(tuple_rep)
            chess_board.remove_piece_at(pawn - 9)
            chess_board.set_piece_at(pawn, chess.Piece.from_symbol('P'))
            
    return new_boards, new_tuples


'''
* As pawns can be exchanged to other figures and this cannot be undone this exchange can also not be reversed
* through the pseudo legal moves.
* To mitigate this a pawn will be placed in the top row in the spot of a queen
'''
def replace_queen_with_pawn(chess_board, used_boards, s_n1_tuples, queen_positions):
    new_boards = []
    new_tuples = set()
    
    for square in queen_positions:
        chess_board.remove_piece_at(square)
        chess_board.set_piece_at(square - 9, chess.Piece.from_symbol('P'))
        if chess_board.is_valid():
            tuple_rep = (chess_board.turn,chess_board.__str__())
            if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples:
                new_boards.append(chess_board.copy())
                new_tuples.add(tuple_rep)
        chess_board.remove_piece_at(square - 9)
        chess_board.set_piece_at(square, chess.Piece.from_symbol('Q'))
    return new_boards, new_tuples
        

def find_pawns(chess_board):
    result = []
    for pawn in chess_board.pieces(chess.PAWN, True):
            result.append(chess.square_name(pawn))
    return result

Ein Problem, das bei der Verwendung der rückwärts-Analyse auftritt, liegt in dem Szenario: 
"König und Bauer gegen König". Dieses Szenario beinhaltet nämlich die Umwandlung des Bauerns in eine andere 
Figur (Dame, Turm, Läufer, Springer). Da in den meisten Fällen die Umwandlung in eine Dame am sinnvollsten ist, 
wurde, wenn der Nutzer einen Bauern als Figur zur Berechnung eingetragen hat, dieser zuvor durch eine Dame ersetzt.
Die sich nun in der Situation befindenden Damen müssen, wo möglich, wieder durch Bauern ersetzt werden. 


Die Funktion ``check_top_row_for_queen`` überprüft ob ein solcher Tausch möglich ist. Sie erhält als Parameter ein 
``board``, für welches die Felder der obersten Reihe (Sichtweise weiß) überprüft und jedes 
zurückgegeben wird, auf dem sich eine Dame befindet.

In [445]:
def check_top_row_for_queen(board):
    return_list = []
    for i in range(56, 64):
        if board.piece_type_at(i) == chess.QUEEN:
            return_list.append(i)

    if len(return_list) > 0:
        return return_list
    else:
        return False
    

In [446]:
'''
* We can choose every turn white does, but not for black.
* If one particular move from black brings the board from S_n to S_n-1 it is not guaranteed that every move
* black can do does so as well.
* To mitigate this only boards where every move ends up in a lower denomination of S will be added to the currently
* calculated one.
* For efficiency an ascii representation is carried through all calculations.
'''
def check_black_determinism(s_n1, used_boards):
    s_n1_tmp = []
    s_n1_tuples_tmp = set()
    
    for chess_board in s_n1:
        include = True
        if not chess_board.turn:
            for move in chess_board.legal_moves:
                chess_board.push(move)
                tuple_rep = (chess_board.turn,chess_board.__str__())
                chess_board.pop()
                if tuple_rep not in used_boards:
                    include = False
        if include:
            s_n1_tmp.append(chess_board)
            s_n1_tuples_tmp.add((chess_board.turn,chess_board.__str__()))
    
    return s_n1_tmp, s_n1_tuples_tmp

Export in Datei
Nach erfolgreicher Berechnung werden die FENs der Boards in eine Datei geschrieben, um eine spätere Auswertung zu ermöglichen.

In [447]:
def create_empty_file(filename):
    f = open("S_n_Results/" + filename + ".preConvert", "w")
    f.write("")
    f.close()

def append_to_file(s_n, filename):
    s_n_ascii = []
    for board in s_n:
        s_n_ascii.append(board.fen())

    f = open("S_n_Results/" + filename + ".preConvert", "a")
    f.write("\n")
    f.write(json.dumps(s_n_ascii))
    f.close()

def convert_file(filename):
    s_n_seq_ascii = []
    f = open("S_n_Results/" + filename + ".preConvert", "r")
    lines = f.readlines()
    first = True
    for line in lines:
        # First line is empty
        if first:
            first = False
            continue

        tmp_list = []
        tmp = json.loads(line)

        for board in tmp:
            tmp_list.append(board)
        s_n_seq_ascii.append(tmp_list)
    f.close()

    f = open("S_n_Results/" + filename + ".chessAI", "w")
    f.write(json.dumps(s_n_seq_ascii))
    f.close()

    if os.path.exists("S_n_Results/" + filename + ".preConvert"):
        os.remove("S_n_Results/" + filename + ".preConvert")

Mit den zuvor definierten Funktionen ist es nun möglich alle $S_n$ zu bestimmen. Hierfür wird die Funktion ``one_step_away`` solange aufgerufen, bis sich die Menge $S$ nicht mehr verändert (Kann auch leer sein).

In [448]:
def run(s_n, used_boards, n, user_wants_pawn, filename):
    if n == 0:
        append_to_file(s_n, filename)

    while True:
        n += 1
        s_n, used_boards = previous_states(s_n, used_boards, n, user_wants_pawn)
        if not s_n: #an empty list is false
            break
        append_to_file(s_n, filename)

    print("Done")
    print(str(n) + " S-Lists calculated")

    convert_file(filename)

    print("File converted")

def run_from_start(user_supplied_pieces, filename):
    user_wants_pawn = chess.Piece.from_symbol("P") in user_supplied_pieces
    
    s_0, used_boards = setup_boards(user_supplied_pieces)

    create_empty_file(filename)
    run(s_0, used_boards, 0, user_wants_pawn, filename)

In [449]:
def resume_from_file(filename):
    used_boards = set()
    s_n = []
    count = -1

    f = open("S_n_Results/" + filename + ".preConvert", "r")
    lines = f.readlines()
    first = True
    for line in lines:
        # First line is empty
        if first:
            first = False
            continue

        tmp_list = []
        tmp_set = set()
        tmp = json.loads(line)

        for fen in tmp:
            chess_board = chess.Board(fen)
            tmp_list.append(chess_board)
            tmp_set.add((chess_board.turn,chess_board.__str__()))
        s_n = tmp_list
        used_boards |= tmp_set
        count += 1
    f.close()
    print("Starting at S" + str(count) + " (" + str(len(s_n)) + " Boards)")
    
    # Adding a few Pawns to the result only slightly increases the file_size, and the value of user_wants_pawn is \
    # currently not stored
    run(s_n, used_boards, count, True, filename)

## Konfigurations Variablen

Für die Bestimmung der gewonnenen Spielbretter, müssen nun die Spielfiguren angegeben werden, für die die
Endspiel-Datenbank berechnet werden soll. Hierfür werden in der Liste ``pieces_to_place`` alle Figuren aufgeführt.
Die Einträge der Liste werden als Tupel bestehend aus Figur und Farbe gespeichert.
Bsp.: ``(chess.KING, chess.WHITE)``
Die Reihenfolge oder Position der Figur ist für die Berechnung irrelevant.
Diese wird erst im nächsten Schritt (der Auswertung) benötigt.

Der ``Filename`` wird für das Ergebnis verwendet.

In [450]:
WHITE_PIECES_TO_PLACE = [chess.Piece.from_symbol('R')]

FILENAME = "S_n_seq_rook_test"

In [451]:
run_from_start(WHITE_PIECES_TO_PLACE, FILENAME)

Done with S33
Done
33 S-Lists calculated
File converted


In [452]:
#resume_from_file(FILENAME)